# Computing temperature properties and mixing length for the DIMES C mooring data

In [1]:
import pickle
import numpy as np
import pandas as pd
import xarray as xr
from scipy.io import loadmat
import gsw
import csaps
import warnings

## Define which data set to use for the relation between $\phi_{1500}$ and the cross-stream distance

In [2]:
data_source = 'SR1b' # can be 'SR1b', 'CMEMS', 'SOSE', or 'Argo'

if data_source == 'SR1b':
    fname = 'data/SR1b_section/pp_dist_sr1b.pkl'
elif data_source == 'CMEMS':
    fname = 'data/CMEMS_reanalysis/pp_dist_cmems.pkl'
elif data_source == 'SOSE':
    fname = 'data/SOSE_reanalysis/pp_dist_sose.pkl'
elif data_source == 'Argo':
    fname = 'data/Argo_data/pp_dist_argo.pkl'
else:
    raise ValueError("Invalid data source. Choose from 'SR1b', 'CMEMS', 'SOSE', or 'Argo'.")

with open(fname, 'rb') as f:
    pp_dist_func = pickle.load(f)

## Read in & preprocess mooring data

In [3]:
# read in mooring data
mooring_c = loadmat('data/DIMES_moorings/mooring_c.mat')['mooring_c']
pressure = mooring_c['P'][0][0][0]

In [4]:
# downsampling to daily values

def downsample(input_array, factor):
    # factor says how much to downsample, i.e. to go from quarter hourly to days we downsample by 24*4=96
    shape = input_array.shape
    output = np.zeros((shape[0], shape[1]//factor))
    for i in range(shape[0]):
        output[i,:] = np.nanmean(input_array[i,:].reshape(-1, factor), 1)
        
    return output

downsampling_factor = 96          # 24 * 4, quarterly measurements

with warnings.catch_warnings(): # this will suppress all warnings in this block
    warnings.simplefilter("ignore")
    mooring_c['CT'][0][0] = downsample(mooring_c['CT'][0][0], downsampling_factor)
    mooring_c['SA'][0][0] = downsample(mooring_c['SA'][0][0], downsampling_factor)
    mooring_c['gamma'][0][0] = downsample(mooring_c['gamma'][0][0], downsampling_factor)
    mooring_c['U'][0][0] = downsample(mooring_c['U'][0][0].T, downsampling_factor)
    mooring_c['V'][0][0] = downsample(mooring_c['V'][0][0].T, downsampling_factor)
     
length_of_array = mooring_c['CT'][0][0].shape[1]

# Conversion of dates from MATLAB format to readable format using pandas
# The value 719529 is the datenum value of the Unix epoch start (1970-01-01), which is the default origin for pd.to_datetime().
dates_c = pd.to_datetime(mooring_c['DATES'][0][0].reshape(42048) - 719529, unit='D')[::downsampling_factor]

In [5]:
CT = mooring_c['CT'][0][0]
SA = mooring_c['SA'][0][0]
gamma = mooring_c['gamma'][0][0]

In [6]:
# interpolate U and V for all moorings (they are only measured at a few pressure levels)

preslevels_c = np.array([453.2, 504.4, 555.1, 602.4, 1233.9, 1332.7, 1888.9, 1987.5, 2085.9, 2190.6, 3434.9, 3637.6])
preslevels_others = np.array([453.2, 550, 1200, 2000, 3500])

U_c = gsw.pchip_interp(preslevels_c, mooring_c['U'][0][0].T, pressure, axis=1).T
V_c = gsw.pchip_interp(preslevels_c, mooring_c['V'][0][0].T, pressure, axis=1).T

U = U_c
V = V_c

In [7]:
# calculate baroclinic streamfunctions and use spatial data to deduce distance from southern edge of section

gpan_c  = -gsw.geo_strf_dyn_height(mooring_c['SA'][0][0], mooring_c['CT'][0][0], mooring_c['P'][0][0][0])
phi1500_c  = gpan_c[105] - gpan_c[5]
phi1500 = phi1500_c  # for now only use center mooring, because it has the most data
distfromphi = pp_dist_func(phi1500)

## Linear interpolation of variables on desired neutral surfaces

In [8]:
win_width = 0.02
igamma = np.arange(27.14, 28.21, win_width)
surf_press = np.zeros((len(igamma),len(distfromphi)))
surf_CT = np.zeros((len(igamma),len(distfromphi)))
surf_SA = np.zeros((len(igamma),len(distfromphi)))
surf_U = np.zeros((len(igamma),len(distfromphi)))
surf_V = np.zeros((len(igamma),len(distfromphi)))

for j in range(len(distfromphi)):
    with warnings.catch_warnings(): # this will suppress all warnings in this block
        warnings.simplefilter("ignore")
        for i in range(len(igamma)):
            icyc = np.where((gamma[:,j] <= igamma[i] + win_width/2) & (gamma[:,j] >= igamma[i] - win_width/2))[0]
            surf_press[i,j] = np.nanmean(pressure[icyc])
            surf_CT[i,j] = np.nanmean(CT[icyc,j])
            surf_SA[i,j] = np.nanmean(SA[icyc,j])
            surf_U[i,j] = np.nanmean(U[icyc,j])
            surf_V[i,j] = np.nanmean(V[icyc,j])

## Calculate mean and rms of temperature and salinity plus their gradients on each neutral surface

In [9]:
# fit splines on isoneutral surfaces to determine mean relationships
pp_CT = [[] for i in range(len(igamma))]  # initialize array to fill with spline fits
pp_SA = [[] for i in range(len(igamma))]

for j in range(len(igamma)):  # only rows where there are actually non-NaNs, see surf_CT[77,:][~np.isnan(surf_CT[77,:])]
    # fit a smoothing spline through relationship
    idx_x = np.argsort(distfromphi)#[:-np.count_nonzero(np.isnan(distfromphi))]
    idx_y = np.argwhere(~np.isnan(surf_CT[j,:])) #non-nan indices in y   

    new_idx = []
    for i in range(len(idx_x)):
        if idx_x[i] in idx_y:
            new_idx.append(idx_x[i])
    new_idx = np.asarray(new_idx)

    # now do spline fit
    smooth = 1e-14
    pp_CT[j] = csaps.csaps(distfromphi[new_idx], surf_CT[j,:][new_idx], smooth=smooth)
    pp_SA[j] = csaps.csaps(distfromphi[new_idx], surf_SA[j,:][new_idx], smooth=smooth)

/nethome/5664187/miniconda3/envs/carlo/lib/python3.13/site-packages/csaps/_sspumv.py:301: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  u = la.spsolve(a, b)


In [10]:
# calculate "background" meridional theta gradient and rms theta anomaly for each isopycnal using a running meridional window

#   Define a meridional distance grid, and the width of the running window
idist     = np.arange(390e3,680e3,10e3)
win_width = 40e3

pressmean     = np.zeros((len(igamma), len(idist)))
SAbar         = np.zeros((len(igamma), len(idist)))
thetabar      = np.zeros((len(igamma), len(idist)))
dthetabardy   = np.zeros((len(igamma), len(idist)))
rms_theta     = np.zeros((len(igamma), len(idist)))


for j in range(len(igamma)):
    dthetabardy[j,:] = np.gradient(pp_CT[j](idist),idist)
    with warnings.catch_warnings(): # this will suppress all warnings in this block
        warnings.simplefilter("ignore")
        for i in range(len(idist)):
            icyc = np.where((distfromphi <= idist[i] + win_width/2) & (distfromphi >= idist[i] - win_width/2))[0]

            pressmean[j,i] = np.nanmean(surf_press[j,icyc])
            thetabar[j,i] = pp_CT[j](idist[i])
            SAbar[j,i]    = pp_SA[j](idist[i])
            
            if len(icyc) > 1:
                rms_theta[j,i] = np.nanstd(surf_CT[j,icyc] - pp_CT[j](distfromphi[icyc]))
            else:
                rms_theta[j,i] = np.nan

SAbar[np.isnan(pressmean)] = np.nan
thetabar[np.isnan(pressmean)] = np.nan
dthetabardy[np.isnan(pressmean)] = np.nan
rms_theta[np.isnan(pressmean)] = np.nan

## Calculate eddy velocity scale on each neutral surface

In [11]:
ue = np.zeros((len(igamma), len(idist))) # eddy velocity scale
upar = np.zeros((len(igamma), len(idist))) # along-stream velocity

for j in range(len(igamma)):
    for i in range(len(idist)):
        icyc = np.where((distfromphi <= idist[i] + win_width/2) & (distfromphi >= idist[i] - win_width/2))[0]
        vel_vec = np.array([surf_U[j,icyc],surf_V[j,icyc]])
        vel_mean = np.nanmean(vel_vec,axis=1) # time-mean velocity vector
        nv = np.array([-vel_mean[1],vel_mean[0]]) # perpendicular vector to time-mean velocity
        n = nv / np.linalg.norm(nv) # normalize
        ucross = surf_U[j,icyc]*n[0] + surf_V[j,icyc]*n[1] # compute u . n for all data points in this window
        ue[j,i] = np.nanstd(ucross)
        p = vel_mean / np.linalg.norm(vel_mean) # normalize vector parallell to time-mean velocity
        upari = surf_U[j,icyc]*p[0] + surf_V[j,icyc]*p[1]
        upar[j,i] = np.nanmean(upari)

/tmp/ipykernel_56955/1991924759.py:8: RuntimeWarning: Mean of empty slice
  vel_mean = np.nanmean(vel_vec,axis=1) # time-mean velocity vector
/nethome/5664187/miniconda3/envs/carlo/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_56955/1991924759.py:15: RuntimeWarning: Mean of empty slice
  upar[j,i] = np.nanmean(upari)


## Calculate mixing length and eddy diffusivity

In [12]:
# mixing efficiency parameter
gamma_mix = 0.35

# based on temperature
dthetabardy_limited = dthetabardy.copy()
dthetabardy_limited[np.where(abs(dthetabardy_limited) < 1e-7)] = 1e-7
Lmix = rms_theta / np.abs(dthetabardy_limited) 
Lmix[np.where(Lmix == 0)] = np.nan

gamma_mix = 0.35 # mixing efficiency parameter
eddy_diff = gamma_mix * ue * Lmix

## Saving the output as a function of cross-stream distance and neutral density

In [13]:
ds = xr.Dataset({"pressmean": (["gamma", "dist"], pressmean),
                 "thetabar": (["gamma", "dist"], thetabar),
                 "dthetabardy": (["gamma", "dist"], dthetabardy_limited),
                 "rms_theta": (["gamma", "dist"], rms_theta),
                 "SAbar": (["gamma", "dist"], SAbar),
                 "ue": (["gamma", "dist"], ue),
                 "upar": (["gamma", "dist"], upar),
                 "Lmix": (["gamma", "dist"], Lmix),
                 "eddy_diff": (["gamma", "dist"], eddy_diff)},
                coords={"gamma": igamma, "dist": idist})
ds.to_netcdf('data/variables_from_moorings_'+data_source+'-dist_gamma.nc')

## Now transform to more intuitive coordinates: latitude and depth

In [14]:
# transform cross-stream distance to latitude
with open('data/pp_dist_lat.pkl', 'rb') as f:
    pp_dist_lat = pickle.load(f)
ilat = pp_dist_lat(idist)  # latitude values corresponding to idist

# compute depth from pressure, for both moorings and section data
zmean = np.zeros(np.shape(pressmean))
for i in range(np.shape(pressmean)[1]):
    zmean[:,i] = gsw.z_from_p(pressmean[:,i], ilat[i])*-1

ds_section_depth = xr.open_dataset('data/variables_from_section-lat_depth.nc')
depth_section = ds_section_depth.depth

In [15]:
# interpolate data from (Y, gamma) to (Y, depth) coordinates (gamma-levels to z-levels)
def convert_to_zlevs(data):
    z_levels = depth_section.values  # use the depth from the section data
    Z = len(z_levels)
    L = len(idist)

    data_on_press = np.full((Z, L), np.nan)
    for i in range(L):
        z_col = zmean[:,i]
        d_col = data[:,i]

        # mask NaNs to avoid interpolation errors
        valid = np.isfinite(z_col) & np.isfinite(d_col)
        if np.count_nonzero(valid) < 2:
            continue  # Not enough points to interpolate

        # if pressure is not monotonic, sort it
        z_sorted = z_col[valid]
        d_sorted = d_col[valid]
        sort_idx = np.argsort(z_sorted)
        z_sorted = z_sorted[sort_idx]
        d_sorted = d_sorted[sort_idx]

        # interpolate to the pressure levels
        data_on_press[:, i] = np.interp(z_levels, z_sorted, d_sorted, left=np.nan, right=np.nan)
    return data_on_press, z_levels

# convert variables to z levels
thetabar_z, iz = convert_to_zlevs(thetabar)
dthetabardy_limited_z, _ = convert_to_zlevs(dthetabardy_limited)
rms_theta_z, _ = convert_to_zlevs(rms_theta)
SAbar_z, _ = convert_to_zlevs(SAbar)
ue_z, _ = convert_to_zlevs(ue)
upar_z, _ = convert_to_zlevs(upar)
Lmix_z, _ = convert_to_zlevs(Lmix)
eddy_diff_z, _ = convert_to_zlevs(eddy_diff)

dthetabardy_limited_z[np.where(abs(dthetabardy_limited_z) < 1e-7)] = 1e-7

In [16]:
ds = xr.Dataset({"thetabar": (["depth", "lat"], thetabar_z),
                 "dthetabardy": (["depth", "lat"], dthetabardy_limited_z),
                 "rms_theta": (["depth", "lat"], rms_theta_z),
                 "SAbar": (["depth", "lat"], SAbar_z),
                 "ue": (["depth", "lat"], ue_z),
                 "upar": (["depth", "lat"], upar_z),
                 "Lmix": (["depth", "lat"], Lmix_z),
                 "eddy_diff": (["depth", "lat"], eddy_diff_z)},
                coords={"depth": iz, "lat": ilat})
ds.to_netcdf('data/variables_from_moorings_'+data_source+'-lat_depth.nc')